In [337]:
from datetime import datetime 
import pandas as pd
import numpy as np
import re
import emoji

# The format string should match the format of the date and time string
# In this case, the date and time string is in the format 'MM/DD/YY, HH:MM:SS AM/PM'
date_time_format = '%m/%d/%y, %I:%M:%S %p'
date_time_list = []
name_list = []
text_list = []
total_file = []

file = r"C:\Users\markkw\Downloads\WhatsApp Chat\_chat.txt"
with open(file, encoding="utf8") as f:
    for line in f.readlines():
        total_file.append(line)
        if line[0] == "[":
            date_time_list.append(datetime.strptime(line.split(']', 1)[0].replace('[', ''), date_time_format))
            name_list.append(line.split(']', 1)[1].split(":",1)[0])
            if ":" in line.split(']', 1)[1]:
                text_list.append(line.split(']', 1)[1].split(":",1)[1].replace("\n", ""))
            else: 
                text_list.append("-")

df = pd.DataFrame()
df['date'] = date_time_list
df['Person'] = name_list
df['Message'] = text_list
df = df[df['Message'] != "-"]
pattern = re.compile(r"|".join(map(re.escape, emoji.EMOJI_DATA)))

df["Emojis"] = df["Message"].apply(lambda x: "".join(pattern.findall(x)))
df["Sentence"] = df["Message"].apply(lambda x: pattern.sub("", x))

# start_date = '2022-01-01 00:01:53'
# end_date = '2022-12-10 16:08:33'
# Select DataFrame rows between two dates
df = df[(df['date'] > '2022-1-1') & (df['date'] <= '2022-12-30')]



In [408]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
import re

def sentiment_analyzer(row):
    score = analyser.polarity_scores(str(row))
    
    return score['compound']

def count_emojis(row):
    return len(row)

def count_char(row):
    return len(row.replace(" ", ""))

def count_words(row):
    return len(row.split(" ")) - 1 

def count_ha(row):
    if row.count('haha') == 1:
        return 2
    elif row.count('hahaha') == 1:
        return 3
    elif row.count('hahah') == 1:
        return 2.5
    else:
        return row.count('haha') * 2

def count_quest(row):
    return row.count('?')

def count_upper(row):
    return sum(1 for char in row if char.isupper())

def count_schelden(row):
    aantal = []
    aantal.append(row.count('kut'))
    aantal.append(row.count('Kut'))
    aantal.append(row.count('KUT'))
    aantal.append(row.count('gvd'))
    aantal.append(row.count('GVD'))
    aantal.append(row.count('godverdomme'))
    aantal.append(row.count('kloten'))
    aantal.append(row.count('klote'))
    aantal.append(row.count('tyfus'))
    aantal.append(row.count('tering'))
    aantal.append(row.count('Tering'))
    aantal.append(row.count('Tyfus'))
    aantal.append(row.count('Kutzooi'))
    aantal.append(row.count('Kanker'))
    aantal.append(row.count('kanker'))
    aantal.append(row.count('KANKER'))
    return sum(aantal)

def langste_woord(row):
    # split the sentences into a list of words
    words = row.split()

    # sort the list of words by length
    words.sort(key=len, reverse=True)

    # return the longest word
    try:
        if len(words[0]) == 194:
            print(row)
        return len(words[0])
    except:
        return np.nan

In [356]:
df_index = df.set_index('date')

In [357]:
df_night = df_index.between_time('0:00', '6:45')
df_ochtend = df_index.between_time('6:00', '12:00')

In [405]:
df['Person'] = df['Person'].replace(' \u202a+31\xa06\xa055442275\u202c', " Chris 'netjes' de Kwaasteniet")
df['Person'] = df['Person'].replace(" Chris 'netjes' de Kwaasteniet", "Chris 'netjes' de Kwaasteniet")
df['Person'] = df['Person'].replace(' \u200eYou', " Mark de Kw")
df['Person'] = df['Person'].replace(' Mark de Kw', "Mark de Kw")
df['Emojis_count'] = df['Emojis'].map(lambda x: count_emojis(x))
df['character_count'] = df['Sentence'].map(lambda x: count_char(x))
df['word_count'] = df['Sentence'].map(lambda x: count_words(x))
df['haha_count'] = df['Sentence'].map(lambda x: count_ha(x))
df['quest_count'] = df['Sentence'].map(lambda x: count_quest(x))
df['upper_count'] = df['Sentence'].map(lambda x: count_upper(x))
df['schelden'] = df['Sentence'].map(lambda x: count_schelden(x))
df['langste_woord'] = df['Sentence'].map(lambda x: langste_woord(x))
df['haha_count'] = df['haha_count'].replace(0, np.nan)
df['Sentiment'] = df_test_en['Sentence'].iloc[-7520:].values
df['Sentiment_score'] = df['Sentiment'].map(lambda x: sentiment_analyzer(x))
df['Sentence'].to_excel("chat_excel.xlsx")

# df['Sentiment'] = df.apply(lambda row : sentiment_analyzer(row), axis=1)
# df['Emojis'].value_counts()

In [ ]:
df_test_en = pd.read_excel("chat_excel_en_sent.xlsx")

In [406]:
df_data = pd.DataFrame(index= df.iloc[1:,:].Person.value_counts().index, columns= {"Berichten": df.iloc[1:,:].Person.value_counts().values})

In [407]:
df_data['Nacht'] = df_night.iloc[1:,:].Person.value_counts()
df_data['Ochtend'] = df_ochtend.iloc[1:,:].Person.value_counts()
df_data['Vragen'] = df.iloc[1:,:].groupby(by="Person").mean().quest_count
df_data['Sentiment'] = df.iloc[1:,:].groupby(by="Person").mean().Sentiment_score
df_data['Sentiment_Tot'] = df.iloc[1:,:].groupby(by="Person").sum().Sentiment_score
df_data['Woorden'] = df.iloc[1:,:].groupby(by="Person").mean().word_count
df_data['Woorden_Tot'] = df.iloc[1:,:].groupby(by="Person").sum().word_count
df_data['Lachen'] = df.iloc[1:,:].groupby(by="Person").mean().haha_count
df_data['Lachen_Tot'] = df.iloc[1:,:].groupby(by="Person").sum().haha_count
df_data['emoji'] = df.iloc[1:,:].groupby(by="Person").mean().Emojis_count
df_data['emoji_Tot'] = df.iloc[1:,:].groupby(by="Person").sum().Emojis_count
df_data['Langste_Bericht'] = df.iloc[1:,:].groupby(by="Person").max().word_count
df_data['Langste_Woord'] = df.iloc[1:,:].groupby(by="Person").max().langste_woord
df_data['Gefrustreerd'] = df.iloc[1:,:].groupby(by="Person").mean().schelden
df_data['Gefrustreerd_Tot'] = df.iloc[1:,:].groupby(by="Person").sum().schelden


C:\Users\markkw\AppData\Local\Temp\ipykernel_17128\3805814710.py:12: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  df_data['Langste_Bericht'] = df.iloc[1:,:].groupby(by="Person").max().word_count
C:\Users\markkw\AppData\Local\Temp\ipykernel_17128\3805814710.py:13: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  df_data['Langste_Woord'] = df.iloc[1:,:].groupby(by="Person").max().langste_woord


In [309]:
lengths = df["Sentence"].str.len()
argmax = np.where(lengths == lengths.max())[0]
print(df.iloc[argmax].values)

[[Timestamp('2022-03-28 11:39:20') " Steven 'joppie' Oudejans"
  " Even een gek vraagje, ken jij wellicht iemand die zich bezighoudt met huurrecht? We kregen gister een beetje een vreemd belletje van onze huisbaas, zij is zelf Italiaans, heeft het huis gekocht toen ze hier woonde en verhuurd het nu aan ons en woont zelf in Doha. We hebben inmiddels een contract voor onbepaalde tijd, maar ze belde gister met een verhaal dat de bank erachter is gekomen dat zij als niet-Nederlandse staatsburger zelf in het buitenland woont en een huis hier verhuurd, dus dat zij 1. het huis moet verkopen of 2. een contract met een maximale duratie van 5 jaar kan aanbieden onder een 'diplomatenclausule'. Ze wilt ons nu dus een nieuw contract laten tekenen waarmee we met terugwerkende kracht nog 4 jaar hier kunnen wonen en zij gedurende de hele duratie van dat contract kan aangeven dat ze terug wilt verhuizen naar Nederland en ons dus ff binnen 3 maanden eruit kan kicken. Voelt niet heel chill en volgens mij

In [38]:
df['Sentence'].to_excel("chat_excel.xlsx")

In [39]:
df_test_en = pd.read_excel("chat_excel_en_sent.xlsx")

In [40]:
df_test_en

,Sentence
0,‎Messages and calls are end-to-end encrypted....
1,"Not sure yet, just ask :)"
2,Hahahaha in Overwatch there is now a Dutch hero
3,Including The voice lines:
4,And when he pops his ult: THE UNIVERSE SINGS ...
...,...
32466,Amsterdam substitute force forced children to...
32467,vibes
32468,Employees too short cock
32469,Hahahahahaha
